# Predicting the occupancies of Belgian trains

## 0. Tips and tricks Jupyter notebooks

* Export your notebooks locally: File -> Download as -> html/ipynb (do both!) (NOTE: don't trust the virtual environment, always download your latest version locally!)

* Shift + Enter to run a cell (or play button)

* TAB for code completion!

* Shells can be Code but also Markdown! (dropdown menu on top allows you to choose)

* Cell -> Run All might come in useful

* In case of problems you can always Kernel -> Restart

* In the Jupyter start window you have a 'new' button on the upper right => to open a terminal, a new python3 notebook, etc.

* You cannot upload very large files (>3GB) via the upload in Jupyter (bug), the datasets will always be available via een public Amazon url, so use wget or scp to get your data in /mnt on the virtual wall!


## 1. Data Preparation

### Essential Libraries for Python Data Science

In [ ]:
#vector/matrix library
import numpy as np
#data frame library (similar to R)
import pandas as pd

#visualization library
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#regular expression library for data cleasning
import re

### Detour: Get up to speed with python / pandas

* If you are new to python have a look at the practicelab.ipynb in the github repo
* If you are new to pandas take some time for:
    - <a href="http://pandas.pydata.org/pandas-docs/stable/10min.html"> Pandas basics in 10 minutes </a>
    - <a href="http://pandas.pydata.org/pandas-docs/stable/visualization.html"> Pandas for data visualizations </a>
    - Use the pandas cheat sheet in the github repo
    
    
#### TIP: Pandas dataframes are immutable, so appending a row creates a copies the old df in the new one => creating a df like this is an O(N^2) operation (I learnt this the hard way)    
 


### 1a. Dataset characteristics

1. Read the train and test datasets
2. How many records and how many features are in train and test sets?
3. What is the range of the querytime column (earliest + latest date)
4. Merge training and test data (data cleansing will be the same for both)
5. Drop columns querytype and user_agent since they contain no useful info, this will speedup future df calculations

In [ ]:
#1
path_train = <FILL_IN>
path_test  = <FILL_IN>

In [ ]:
#warning the input files are slightly different format, training data is in new-line delimited json, 
#testdata in regular json
df_train = pd.read_json(<FILL_IN>)
df_test = pd.read_json(<FILL_IN>)

#to have a look at a dataframe just use head or tail
df_train.head(n=5)

In [ ]:
df_test.head(n=5)

In [ ]:
#2
print("Number of records in training set: " + str(<FILL_IN>))
print("Number of records in test set: " + str(<FILL_IN>))


In [ ]:

print("Features training data: " + str(<FILL_IN>))
print("Features test data: " + str(<FILL_IN>))

In [ ]:
#3
print('Querytime min (training): ' + str(<FILL_IN>))
print('Querytime max (training): ' + str(<FILL_IN>))
print()
print('Querytime min (test): ' + str(<FILL_IN>))
print('Querytime max (test): ' + str(<FILL_IN>))

In [ ]:
#4
#merge train and test data AND reset the index
dataset_v1 = <FILL_IN>


In [ ]:
#5
dataset_v2 = <FILL_IN>

### 1b. Parsing the columns

HINT: very often you will take a column an 'apply' a transformation to it. Look up the pandas syntax for apply()!

1. In the id column replace the NaNs with -1
2. Select a row by id and have a closer look at the post column (json object)
    * check multiple rows since the post object doesn't always have the same fields
3. Iterate over the frame while keeping track of the possible fields in post => what is the set of ps
4. Write a function to extract a field of the json object 
5. For every field in the json objects create an additional column, finally drop the post column
6. The query time column is now interpreted as a string => convert to datetime, the id column can be cast to integer



    


In [ ]:
#1
transformed_column = <FILL_IN>
dataset_v2['id'] = transformed_column


In [ ]:
#2
row = <FILL_IN>
post_obj = row['post']
post_obj

In [ ]:
#3
   
unique_keys = <FILL_IN>

In [ ]:
#4
def extract_field(json, field):
    <FILL_IN>

In [ ]:
#5
dataset_v3 = <FILL_IN>    

In [ ]:
#6
dataset_v3['querytime'] = <FILL_IN>
dataset_v3['id'] = <FILL_IN>
dataset_v3.dtypes

### 1.3 And some more cleansing

- TASK: go over each of the columns, further parse them and pay close attention to NULL values
- HINT: don't mindlessly drop rows with missing data, the dataset has some level of redundancy so pay close attention!

- HINT: Look up the pandas isnull() function
- HINT: use the value_counts() function to check the distribution of values in a column

1. How many NULL values per column?
2. Rework the vehicle column:
    - use the following information on the slides and at: https://nl.wikipedia.org/wiki/Lijst_van_treincategorie%C3%ABn_in_Belgi%C3%AB
    - extract connection type, train series, sequence number and you can also infer the direction of the train
    - keep checking for NULLs, some can be disguised

4. connection
5. date
6. occupancy
7. to, from columns

In [ ]:
#1
<FILL_IN>

In [ ]:
dataset_v3[pd.isnull(dataset_v3['to'])]

In [ ]:
#2
dataset_v3['vehicle'] = <FILL_IN>

#write some helper functions to extract the vehicle features such as train_type, direction, series,..


In [ ]:
#3

In [ ]:
dataset_v4 = dataset_v3.drop(<FILL_IN>)

In [ ]:
dataset_v4['train_series']    = <FILL_IN>
dataset_v4['train_direction'] = <FILL_IN>
dataset_v4['train_type']      = <FILL_IN>

dataset_v5 = dataset_v4.drop(<FILL_IN>)

In [ ]:
#4 


In [ ]:
#5
dataset_v5['occupancy'] = <FILL_IN>

In [ ]:
#6
dataset_v5['from'].describe()

In [ ]:
dataset_v5['to'].describe()

In [ ]:
#study the bad records (not only NULL!)

In [ ]:


dataset_v6 = dataset_v5.drop(<FILL_IN>) #drop bad records in to column
dataset_v7 = dataset_v6.drop(<FILL_IN>) #drop bad records in from column

In [ ]:
dataset_v7.shape

In [ ]:
print(dataset_v7[dataset_v7['id'] == -1].shape)
print(dataset_v7[dataset_v7['id'] != -1].shape)

dataset_v7.to_csv('json_cleaned.csv', header=True, sep=',', index=False)